### MRI Image Classification With CNN (CS584, Please do not re-post on the web!)

In this tutorial, we show you step-by-step how to classify a set of 3D MRI images into two main classes: Non-stroke and Stroke. We describe how you can read the 3D-MRI data and how you can use your own data for learning of a deep architecture like a Convolutional Neural Network (CNN).


### Dataset

The dataset for this project are 202 synthetic image sequences, generated to closely resemble real-world examples of Non-stroke and Stroked brain on Apparent Diffusion Coefficient (ADC) maps; a type of post-processed MRI image sequence (See https://www.youtube.com/watch?v=MMyeQpHLeng&t=6s). Each image sequence is classified by an expert into either Non-stroke or Stroke class (0 v.s. 1, respectively). Starting from the original 202 image sequences, we applied 'Data Augmentation' to increase the dataset size to 808 image sequences of size 128x128x24 (image resolution: 128x128, slice/sequence resolution: 24). Here we only performed basic image rotation (-+5 degrees), flipping, and rotation+flipping. But, you may apply other types of data augmentation to increase the size of your dataset.

* For more information on MRI images see:
https://www.youtube.com/watch?v=mOt2FeGHjaY
https://bmcmedimaging.biomedcentral.com/articles/10.1186/1471-2342-11-2

### Loading the Libraries

Loads all the required libraries and initialize certain parameters related to the Data.

In [ ]:
from __future__ import division
import tensorflow as tf
import numpy as np
import os
import csv
from sklearn.utils import shuffle
import time
from sklearn import metrics

learning_rate = 0.0001

# image dimensions
n_depth = 24
n_input_x = 128
n_input_y = 128
n_classes = 2

Counter_batch = 0
#Maximum iteration for training
max_iter = 10
#Batch size
batch_sz = 5

### Finding the Path to Images

There are two different CSV files one containing the name of each image and the other one contains the corresponding label. In this stage, we find the path of each image and saved the results to "pathDicom3".


In [ ]:
All_Files = []
All_labels = []

with open('/home/class/cs584/stroke_data_augmented_outcomes.csv', 'rb') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
    for row in spamreader:
        All_Files.append(row[0])
        All_labels.append(row[2])


All_Files_202 = []
All_labels_202 = []

with open('/home/class/cs584/stroke_data_outcomes.csv', 'rb') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
    for row in spamreader:
        All_Files_202.append(row[0])
        All_labels_202.append(row[2])

File_exclude_Ind = [7,12,37,40,51,76,79,80,88,94,96,101,102,110,159,164,181,200]


File_exclude = []

for i in range(len(File_exclude_Ind)):
    File_exclude.append(All_Files_202[File_exclude_Ind[i]])


print 'Number of File Exclude: %g' % len(File_exclude)

pathDicom = os.listdir("/home/class/cs584/synthetic_images/")
pathDicom3 = []
item_values = []


for item in pathDicom:
    check = 0
    for i in range(len(File_exclude)):
        if File_exclude[i] in item:
            check = 1
            break
        else:
            check = 0

    if check == 0:
        item2 = '/home/class/cs584/synthetic_images/' + item
        item_values.append(item)
        pathDicom2 = os.listdir(item2)
        substring1 = 'ADC'
        substring2 = 'Apparent_Diffusion_Coefficient_(mm2s)'
        gg = 0
        for i in range(len(pathDicom2)):
            if substring1 in pathDicom2[i]:
                item3 = item2 + '/' + pathDicom2[i]
                gg += 1
                break

            if substring2 in pathDicom2[i]:
                item3 = item2 + '/' + pathDicom2[i]
                gg += 1
                break

        pathDicom3.append(item3)

### Reading of the Dataset

In this step, 3D-MRI images are being read from the text files and the results will be saved in a 3D-matrix. It must be considered that each image is a 3D matrix of size (128x128x24). Finally, this data will be used for learning of Convolutional Neural Network.


In [ ]:
Input_data = []
label = []
count = -1
for item in pathDicom3:
    count = count + 1
    try:
        item_index = All_Files.index(item_values[count])
        label.append(All_labels[item_index])
    except:
        f = 0

n_examples = 100

# Assign the Actual label value
Actual_Label = np.zeros(n_examples, dtype= int)
for i in range(n_examples):
    if label[i] == '1':
        Actual_Label[i] = 1
    else:
        Actual_Label[i] = 0

FeedData = []
Files = []
P_Files = []
for num in range (100):
        for i in range (n_depth):

                f = open(pathDicom3[num] + "/IM-"+str(i+1)+".txt")
                for row in f.readlines():
                    for word in row.split(','):
                       if word != '0\n':
                           if word == '0':
                                Files.append(int(word))
                           else:
                               Files.append(float(word))
                       else:

                           Files.append(0)

                Files_Array = np.array(Files)
                Files_Array = np.float32(Files_Array)

                Files = []
                Files_Image = np.reshape(Files_Array, (n_input_x, n_input_y))
                P_Files.append(Files_Image)

        FeedData.append(P_Files)
        P_Files = []

FeedData1 = np.array(FeedData)


del FeedData
#normalization
size = FeedData1.shape

print size


MAX_Dataset = np.amax(FeedData1)
Normalized_FeedData1 = FeedData1/ MAX_Dataset

Data_Input = FeedData1.reshape((n_examples, n_depth, n_input_x, n_input_y, 1))
# TF Section
del FeedData1

label_data = np.zeros([n_examples, 2])

for i in range(n_examples):
    if label[i] == '1':
        label_data[i, 1] = 1
    else:
        label_data[i, 0] = 1

### Build the Convolutional Layers

In this step, we need to define the various parameters of each layer, such as the number of feature maps (or filters/kernels), parameters of each filter, biases, activation functions and pooling operations (max, average). Note that the filter parameters or weights are the convolutional filters that will be learned during the training process. We then put all of these components together to make a CNN layer. Finally, this procedure has to be repeated to make a CNN with an arbitrary number of layers.


In [ ]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1, seed = 10000)
    return tf.Variable(initial)


def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

# Number of filters at the first Conv layer
N_Conv_layer1 = 5

# Output size of the Fully Connected layer
N_W_fc1 = 50
N_b_fc1 = 50

x = tf.placeholder(tf.float32, [None, n_depth, n_input_x, n_input_y, 1])
y_ = tf.placeholder(tf.float32, [None, n_classes])

W_cv1 = weight_variable([3, 3, 3, 1, N_Conv_layer1])
b_cv1 = bias_variable([N_Conv_layer1])

conv1 = tf.nn.conv3d(x, W_cv1, strides=[1, 1, 1, 1, 1], padding="SAME")
conv1 = tf.nn.bias_add(conv1, b_cv1)
conv1 = tf.nn.relu(conv1)
conv1 = tf.nn.max_pool3d(conv1, ksize=[1, 4, 4, 4, 1], strides=[1, 4, 4, 4, 1], padding="SAME")

# MLP
W_fc1 = weight_variable([6 * 32 * 32 * N_Conv_layer1, N_W_fc1])
b_fc1 = bias_variable([N_b_fc1])

h_pool2_flat = tf.reshape(conv1, [-1, 6 * 32 * 32 * N_Conv_layer1])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

W_fc2 = weight_variable([N_W_fc1, 2])
b_fc2 = bias_variable([2])
y_conv = (tf.matmul(h_fc1, W_fc2) + b_fc2)

#Regularization
reg = (tf.nn.l2_loss(W_fc1) + tf.nn.l2_loss(b_fc1) + tf.nn.l2_loss(W_fc2) + tf.nn.l2_loss(b_fc2))

### Define Optimizer

In this section, we define the loss function and an optimizer for training section. According to the code bellow, the "softmax_cross_entropy" uses as loss function and "AdamOptimizer" for optimization algorithm. The accuracy can be computed based on the third and fourth lines.


In [ ]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y_conv, y_)) + 0.1 * reg
optimizer = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
Y_softmax = tf.nn.softmax(tf.matmul(h_fc1, W_fc2) + b_fc2)

### Training and Validation and Test Set

Before feeding the data into the network, we have to separate entire dataset into these three main sections. 


In [ ]:
Training = []
Testing = []
training_l = []
test_l = []

Training_set = Data_Input[:60,]
training_labels = label_data[:60]
Valid_set = Data_Input[60:80,]
Valid_label = label_data[60:80]
Test_set = Data_Input[80:,]
Test_label = label_data[80:]

del Data_Input
del label_data

init = tf.initialize_all_variables()
saver = tf.train.Saver(max_to_keep=100)

n_batches = int(len(Training_set)/batch_sz)

### Training of the Network

In this section, the data is broken into different mini-batches, then each batch feeds into the network separately. The defined Optimizer optimizes the weight values in each iteration. In this step, the weights will be updated by optimizer according to the defined the loss function. This procedure is called backpropagation.


In [ ]:
training_accuracy = np.zeros(len(range(max_iter))).astype(float)
valid_accuracy = np.zeros(len(range(max_iter))).astype(float)
Test_Accuracy = np.zeros(len(range(max_iter))).astype(float)
val_smooth = np.zeros(len(range(max_iter))).astype(float)
tempt = np.zeros(len(range(max_iter))).astype(float)
AUC0 = np.zeros(len(range(max_iter))).astype(float)
AUC1 = np.zeros(len(range(max_iter))).astype(float)

NUM_THREADS = 120
with tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=NUM_THREADS,inter_op_parallelism_threads=NUM_THREADS,log_device_placement=False)) as session:
    session.run(init)

    for i in xrange(max_iter):
        Xtrain_temp, Ytrain_temp = shuffle(Training_set, training_labels)
        shuf2 = shuffle(range(n_batches))
        n_batches_half = int(n_batches/n_batches)
        for j in xrange(n_batches_half):

            batch1 = Xtrain_temp[shuf2[j] * batch_sz:(shuf2[j] * batch_sz + batch_sz), ]
            batch2 = Ytrain_temp[shuf2[j] * batch_sz:(shuf2[j] * batch_sz + batch_sz), ]


            if j%5 == 0:
                train_accuracy = accuracy.eval(feed_dict={x: batch1, y_: batch2})
                print("Step %d, BatchNumber: %d, training accuracy %g" % (i, j, train_accuracy))

            optimizer.run(feed_dict={x: batch1, y_: batch2})

        n_batches_train = int(len(training_labels) / batch_sz)
        n_batches_test = int(len(Test_label) / batch_sz)
        n_batches_val = int(len(Valid_label) / batch_sz)

        training_acc = np.zeros(len(range(n_batches_train))).astype(float)
        valid_acc = np.zeros(len(range(n_batches_val))).astype(float)
        Test_Acc = np.zeros(len(range(n_batches_test))).astype(float)

        for j in xrange(n_batches_train):
            batch_train1 = Training_set[j * batch_sz:(j * batch_sz + batch_sz), ]
            batch_train2 = training_labels[j * batch_sz:(j * batch_sz + batch_sz), ]
            training_acc[j] = accuracy.eval(feed_dict={x: batch_train1, y_: batch_train2})

        training_accuracy[i] = np.mean(training_acc)

        for j in xrange(n_batches_test):
            batch_test1 = Test_set[j * batch_sz:(j * batch_sz + batch_sz), ]
            batch_test2 = Test_label[j * batch_sz:(j * batch_sz + batch_sz), ]
            start_time = time.time()
            Test_Acc[j] = accuracy.eval(feed_dict={x: batch_test1, y_: batch_test2})
            print("--- %s seconds ---" % (time.time() - start_time))

        Test_Accuracy[i] = np.mean(Test_Acc)

        for j in xrange(n_batches_val):
            batch_val1 = Valid_set[j * batch_sz:(j * batch_sz + batch_sz), ]
            batch_val2 = Valid_label[j * batch_sz:(j * batch_sz + batch_sz), ]
            valid_acc[j] = accuracy.eval(feed_dict={x: batch_val1, y_: batch_val2})

        valid_accuracy[i] = np.mean(valid_acc)


        if i == 0:
            val_smooth[i] = valid_accuracy[i]

        if (i >= 1):
            val_smooth[i] = 0.7 * valid_accuracy[i - 1] + 0.3 * valid_accuracy[i]

        # lable contains the actual class labels
        Actual_Label_Test = Actual_Label[80:]
        AUC = y_conv.eval(feed_dict={x: Test_set, y_: Test_label})
        fpr0, tpr0, thresholds0 = metrics.roc_curve(Actual_Label_Test, AUC[:, 0], pos_label=0)
        fpr1, tpr1, thresholds1 = metrics.roc_curve(Actual_Label_Test, AUC[:, 1], pos_label=1)
        AUC0[i] = metrics.auc(fpr0, tpr0)
        AUC1[i] = metrics.auc(fpr1, tpr1)
        print('============================================================')
        print('Training Accuracy: %g' % training_accuracy[i])
        print('Validation Accuracy: %g' % val_smooth[i])
        print ('Test Accuracy: %g' % Test_Accuracy[i])
        print('AUC Measure Class 0: %g' % AUC0[i])
        print('AUC Measure Class 1: %g' % AUC1[i])
        print("============================================================")
        save_path = saver.save(session, str(i) + "Checkpoint.ckpt")
        print "Fast2"


    np.savetxt("Validation-Fast2.csv", val_smooth, delimiter=",")
    np.savetxt("train-Fast2.csv", training_accuracy, delimiter=",")
    np.savetxt("Test-Fast2.csv", Test_Accuracy, delimiter=",")
    np.savetxt("AUC0.csv", AUC0, delimiter=",")
    np.savetxt("AUC1.csv", AUC1, delimiter=",")    